In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive

In [0]:
import tensorflow as tf
import keras
import os
import numpy
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Dense
from keras.layers import Dropout

In [0]:
train_datagen= ImageDataGenerator()
valid_datagen= ImageDataGenerator()
test_datagen= ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(

    directory='PATH/TO/TRAIN/DATASET',

    target_size=(224, 224),

    color_mode="rgb",

    batch_size=32,

    class_mode="categorical",

    shuffle=True,

    seed=42)


valid_generator = valid_datagen.flow_from_directory(

    directory='PATH/TO/VALID/DATASET',

    target_size=(224, 224),

    color_mode="rgb",

    batch_size=32,

    class_mode="categorical",

    shuffle=True,

    seed=42)

test_generator= test_datagen.flow_from_directory(

   directory='PATH/TO/VALID/DATASET',

   target_size=(224,224),color_mode="rgb",

   class_mode="categorical",shuffle=False,seed=42)


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

STEP_SIZE_TEST = test_generator.n//test_generator.batch_size



In [0]:
base_model =  keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3), alpha=1, include_top=False, weights='imagenet', input_tensor=None, pooling=None, classes=7)

x = base_model.output

x = GlobalAveragePooling2D(name='avg_pool')(x)

x = Dense(1024, activation='relu')(x)

x= Dropout(0.5)(x)

predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = True




In [0]:
model.compile(optimizer='adam',

              loss='categorical_crossentropy',

              metrics=['accuracy'])

In [0]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 24) 648         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 24) 96          Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [0]:
keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

model.fit_generator(generator=train_generator,validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=2,callbacks=[tf.keras.callbacks.TensorBoard('/content/drive/My Drive/Efficient_nets/logs')])

In [0]:
model.save("PATH/TO/SAVE/MODEL/mobilenets_fully_trained.model")

###Fine-tuning the model

In [0]:
from keras.models import load_model
model = load_model("PATH/TO/SAVED/MODEL/mobilenets_fully_trained.model")

model.trainable = True

fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:fine_tune_at]:
    layer.trainable =  False


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
model.fit_generator(generator=train_generator,validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,steps_per_epoch=STEP_SIZE_TRAIN,epochs=5)


In [0]:
model.evaluate_generator(test_generator,steps = STEP_SIZE_TEST, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [0]:
import tensorflow as tf
import numpy as np
converter  = tf.lite.TFLiteConverter.from_keras_model_file("PATH/TO/SAVED/MODEL/mobilenets_fully_trained.model")
converter.allow_custom_ops = True
converter.target_ops = [tf.lite.OpsSet.SELECT_TF_OPS, tf.lite.OpsSet.TFLITE_BUILTINS]

tflite_model  = converter.convert()

open("PATH/TO/SAVE/MODEL/mobilenets_fully_trained.tflite", "wb").write(tflite_model)

##F-1 Score


In [0]:
validation_data_dir = "PATH/TO/DATASET"
import glob
allvalidimgpaths=glob.glob(os.path.join(validation_data_dir, '*/*.jpeg'))
allvalidimgpaths+=glob.glob(os.path.join(validation_data_dir, '*/*.jpg'))
import random
random.shuffle(allvalidimgpaths)

In [0]:
from keras.preprocessing.image import load_img, img_to_array
validation_imgs = [img_to_array(load_img(img, target_size=(224,224))) for img in allvalidimgpaths]

In [0]:
validation_labels= [fn.split('/')[-2] for fn in allvalidimgpaths]

import numpy as np
np.array(validation_imgs)

print(np.shape(validation_imgs))

(221, 224, 224, 3)


In [0]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
#LABELS
le.fit(["black eyed beans","chana dal","kidney beans","masoor dal","moong dal","toor dal","urad dal"])
validation_labels_enc = le.transform(validation_labels)

In [0]:
print(validation_labels[5:9])
print(validation_labels_enc[5:9])

In [0]:
newvalids=validation_imgs[:]

valid_imgs_array=numpy.asarray(validation_imgs)

print(valid_imgs_array.shape)
valid_imgs_array.astype('float32')

valid_imgs_array/=255

test_predictions=model.predict(valid_imgs_array)


In [0]:
import numpy as np
newpreds= np.argmax(test_predictions,axis=1)

from sklearn.metrics import classification_report
#Printing the F1 score
print(classification_report(validation_labels_enc, newpreds, target_names=["black eyed beans","chana dal","kidney beans","masoor dal","moong dal","toor dal","urad dal"]))